In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "notebooks"
    cd(joinpath(@__DIR__, "../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMolMonteCarlo`


In [2]:
folder = "assets/hpc_out/"

"assets/hpc_out/"

In [3]:
for file in readdir(folder)
    mt = split(file, "_")[2]
    num = split(file, "_")[3]
    mkpath(joinpath(folder, "$(mt)_$(num)"))
    mv(joinpath(folder, file), joinpath(folder, "$(mt)_$(num)", file))
end

In [4]:
min_path_by_mol = Dict("6r7m" => "", "6sae" => "", "6sag" => "", "2tmv" => "", "1ei7" => "")
E_min_by_mol = Dict("6r7m" => Inf, "6sae" => Inf, "6sag" => Inf, "2tmv" => Inf, "1ei7" => Inf)
mol_types = ["6r7m", "6sae", "6sag", "2tmv", "1ei7"]

rwm_hot_start_folder = "assets/input/jld2/2_low/"
for mol_type in mol_types
    for file in readdir(rwm_hot_start_folder)
        if  split(file, ".")[end] == "jld2" && split(file, "_")[1] == mol_type
            try
                @load "$rwm_hot_start_folder$file" template_mol template_radii x_init
                realization = MorphoMol.Utilities.get_flat_realization(x_init, template_mol)
                E = MorphoMol.Energies.solvation_free_energy(
                    realization, 
                    length(template_radii),
                    [template_radii; template_radii],
                    1.4,
                    MorphoMol.Energies.get_prefactors(1.4, 0.3665),
                    0.0,
                    0.85
                    )
                if E_min_by_mol[mol_type] > E
                    E_min_by_mol[mol_type] = E
                    min_path_by_mol[mol_type] = "$rwm_hot_start_folder/$file"
                end
            catch e
                println(e)
            end
        end
    end
end

for mol_type in mol_types
    if min_path_by_mol[mol_type] != ""
        @load min_path_by_mol[mol_type] template_mol template_radii x_init
        realization = MorphoMol.Utilities.get_flat_realization(x_init, template_mol)
        E = MorphoMol.Energies.solvation_free_energy(
            realization, 
            length(template_radii),
            [template_radii; template_radii],
            1.4,
            MorphoMol.Energies.get_prefactors(1.4, 0.3665),
            0.0,
            0.85
            )
        template_mol_sim = deepcopy(template_mol)
        x_sim = deepcopy(x_init)
        @load "assets/input/jld2/experimental/2_$(mol_type)_protor.jld2" template_mol template_radii x_init
        println(mol_type, " ", minimum(E), " ", MorphoMol.Utilities.average_offset_distance(template_mol_sim, template_mol, x_sim, x_init))
    end
end


6r7m 8138.924225194982 0.9982893464282008
6sae 8107.279622532144 1.2821182928454404
6sag 8097.730817128498 1.1146356769067605
2tmv 8056.550361360152 21.083570792125695
1ei7 8185.798536861604 24.19202036075877


In [5]:
E_min_by_mol = Dict("6r7m" => Inf, "6sae" => Inf, "6sag" => Inf, "2tmv" => Inf, "1ei7" => Inf)
min_path_by_mol = Dict("6r7m" => "", "6sae" => "", "6sag" => "", "2tmv" => "", "1ei7" => "")
for mol_type in mol_types
    E_min = E_min_by_mol[mol_type]
    min_file_path = ""
    for sf in readdir(folder)
        for file in readdir("$folder$sf")
            if  split(file, ".")[end] == "jld2" && split(file, "_")[2] == mol_type
                try
                    @load "$folder$sf/$file" in_out_data
                    if E_min > minimum(in_out_data.output.Es)
                        E_min = minimum(in_out_data.output.Es)
                        min_file_path = "$folder$sf/$file"
                    end
                catch e
                    println(e)
                end
            end
        end
    end
    min_path_by_mol[mol_type] = min_file_path
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/twZ5D/src/JLD2.jl:298


EOFError()


In [6]:
println(min_path_by_mol)

Dict("6sag" => "", "1ei7" => "assets/hpc_out/1ei7_20/25_1ei7_20_protor_oj_0_0_os_0_85.jld2", "6r7m" => "", "6sae" => "", "2tmv" => "")


In [7]:
for mol_type in ["6r7m", "6sae", "6sag", "2tmv", "1ei7"]
    if min_path_by_mol[mol_type] != ""
        @load min_path_by_mol[mol_type] in_out_data
        template_mol_sim = in_out_data.input.template_mol
        x_sim = in_out_data.output.states[1]
        
        @load "assets/input/jld2/experimental/2_$(mol_type)_protor.jld2" template_mol template_radii x_init
        println(mol_type, " ", minimum(in_out_data.output.Es), " ", MorphoMol.Utilities.average_offset_distance(template_mol_sim, template_mol, x_sim, x_init))
    end
end 

1ei7 8184.615394286413 0.9825672375691238
